<img src="img/sippycup-small.jpg" align="left" style="padding-right: 30px"/>

<h1 style="line-height: 125%">
  SippyCup<br />
  Unit 1: Natural language arithmetic
</h1>

<p>
  Nikita Nangia
  <br>
  Material taken from <a href="http://nlp.stanford.edu/~wcmac/">Bill MacCartney</a>'s SippyCup tutorial.
</p>

This is Unit 1 of the SippyCup codelab. This is one of four parts of a more comprehensive overview of SippyCup. You can find the rest of the tutorial on the [SippyCup github page](https://github.com/wcmac/sippycup).

# 1. Data
## Example inputs

The arithmetic domain is simple and straightforward, and for now our goals are strictly pedagogical.  So a small, artificial sample of inputs will suffice.  Here's a sample of 17 inputs borrowed from the [companion code][] to [Liang & Potts 2015][].

  [companion code]: https://github.com/cgpotts/annualreview-complearning
  [Liang & Potts 2015]: http://www.annualreviews.org/doi/pdf/10.1146/annurev-linguist-030514-125312

In [1]:
inputs = [
    "one plus one",
    "one plus two",
    "one plus three",
    "two plus two",
    "two plus three",
    "three plus one",
    "three plus minus two",
    "two plus two",
    "three minus two",
    "minus three minus two",
    "two times two",
    "two times three",
    "three plus three minus two",
    "minus three",
    "three plus two",
    "two times two plus three",
    "minus four",
]

Note that, even for the arithmetic domain, this sample is quite limited in scope.  It uses only the integers one through four, and it uses only a small number of arithmetic operations.  The [exercises](#arithmetic-exercises) at the end of this unit will challenge you to extend the scope of the problem in various ways.

## Semantic representation <a id="arithmetic-semantic-representation"></a>

Having collected a sample of inputs, the next order of business is to choose a good semantic representation.  After all, the semantic representation is the desired *output* of the semantic parsing system, so the choice of representation will drive many other decisions.

**Our semantic representation should be machine-readable, unambiguous, and easily executable**.  For the domain of natural language arithmetic, a natural choice is to use [binary expression trees][], represented in Python by nested tuples. All of the following are valid semantic representations:

  [binary expression trees]: http://en.wikipedia.org/wiki/Binary_expression_tree

In [2]:
sems = [
    ('+', 1, 1),                # one plus one
    ('-', ('~', 3), 2),         # minus three minus two
    ('-', ('+', 3, 3), 2),      # three plus three minus two
    ('+', ('*', 2, 2), 3),      # two times two plus three
]

It's easy to implement an executor which actually performs the arithmetic calculations described by these semantic representations to return a denotation.

In [3]:
ops = {
    '~': lambda x: -x,
    '+': lambda x, y: x + y,
    '-': lambda x, y: x - y,
    '*': lambda x, y: x * y,
}

def execute(sem):
    if isinstance(sem, tuple):
        op = ops[sem[0]]
        args = [execute(arg) for arg in sem[1:]]
        return op(*args)
    else:
        return sem

Note that the executor is simple, straightforward, deterministic, and doesn't rely on any linguistic knowledge.  This is a sign that we've chosen a good semantic representation!

Let's see the executor in action:

In [4]:
for sem in sems:
    print('{} = {}'.format(sem, execute(sem)))

('+', 1, 1) = 2
('-', ('~', 3), 2) = -5
('-', ('+', 3, 3), 2) = 4
('+', ('*', 2, 2), 3) = 7


It works!

## Example data <a id="arithmetic-example-data"></a>

Now that we've collected a sample of inputs and defined a semantic representation, we can construct a set of *examples* which pair inputs with their target outputs.
The SippyCup codebase defines (in [`example.py`](./example.py)) a simple container class called `Example`.  By importing this class, we can define some examples to guide our development.

<!-- TODO: consider moving all imports to a block at the end of Unit 0. -->

In [6]:
from example import Example

arithmetic_examples = [
    Example(input="one plus one", semantics=('+', 1, 1), denotation=2),
    Example(input="one plus two", semantics=('+', 1, 2), denotation=3),
    Example(input="one plus three", semantics=('+', 1, 3), denotation=4),
    Example(input="two plus two", semantics=('+', 2, 2), denotation=4),
    Example(input="two plus three", semantics=('+', 2, 3), denotation=5),
    Example(input="three plus one", semantics=('+', 3, 1), denotation=4),
    Example(input="three plus minus two", semantics=('+', 3, ('~', 2)), denotation=1),
    Example(input="two plus two", semantics=('+', 2, 2), denotation=4),
    Example(input="three minus two", semantics=('-', 3, 2), denotation=1),
    Example(input="minus three minus two", semantics=('-', ('~', 3), 2), denotation=-5),
    Example(input="two times two", semantics=('*', 2, 2), denotation=4),
    Example(input="two times three", semantics=('*', 2, 3), denotation=6),
    Example(input="three plus three minus two", semantics=('-', ('+', 3, 3), 2), denotation=4),
    Example(input="minus three", semantics=('~', 3), denotation=-3),
    Example(input="three plus two", semantics=('+', 3, 2), denotation=5),
    Example(input="two times two plus three", semantics=('+', ('*', 2, 2), 3), denotation=7),
    Example(input="minus four", semantics=('~', 4), denotation=-4),
]

<!-- TODO: Consider whether to divide these into train examples and test examples. -->

Note that for examples with syntactically ambiguous inputs, our target semantics and denotations reflects a specific choice about how to resolve the ambiguity which accords with the standard [order of operations][].
<!-- TODO: rewrite? -->

  [order of operations]: http://en.wikipedia.org/wiki/Order_of_operations

# 2. Syntactic Parsing
### Define a CFG

In order to build a valid parse tree over an input, we need to know the space of possibilities.  This is the role of the *grammar*, which in SippyCup is a [context-free grammar][] (CFG).  The grammar contains a collection of *rules*, each of which specifies a valid *local subtree*, consisting of a parent and its immediate children.

  [context-free grammar]: http://en.wikipedia.org/wiki/Context-free_grammar

Here's a simple Python class for representing grammar rules.  You can ignore `sem` for now — we're only concerned with syntax at this point.

In [12]:
# defining the rules of the cfg
class Rule:
    def __init__(self, lhs, rhs, sem=None):
        self.lhs = lhs
        self.rhs = tuple(rhs.split()) if isinstance(rhs, str) else rhs
        self.sem = sem

    def __str__(self):
        return 'Rule' + str((self.lhs, ' '.join(self.rhs), self.sem))

We can now write down a few grammar rules for the arithmetic domain.

In [13]:
numeral_rules = [
    Rule('$E', 'one'),
    Rule('$E', 'two'),
    Rule('$E', 'three'),
    Rule('$E', 'four'),
]

operator_rules = [
    Rule('$UnOp', 'minus'),
    Rule('$BinOp', 'plus'),
    Rule('$BinOp', 'minus'),
    Rule('$BinOp', 'times'),
]

compositional_rules = [
    Rule('$E', '$UnOp $E'),
    Rule('$EBO', '$E $BinOp'),  # binarized rule
    Rule('$E', '$EBO $E'),      # binarized rule
]

def arithmetic_rules():
    return numeral_rules + operator_rules + compositional_rules

Note that these rules are in [Chomsky normal form][]. Roughly, CNF requires that every grammar rule have one of two forms:

  [Chomsky normal form]: http://en.wikipedia.org/wiki/Chomsky_normal_form

- In a *unary lexical rule*, the RHS must consist of exactly one word (terminal).
- In a *binary compositional rule*, the RHS must consist of exactly two categories (non-terminals).

Let's define some helper functions which will help us ensure that our __grammars are in CNF.__

In [14]:
def is_cat(label):
    return label.startswith('$')

def is_lexical(rule):
    return all([not is_cat(rhsi) for rhsi in rule.rhs])

def is_binary(rule):
    return len(rule.rhs) == 2 and is_cat(rule.rhs[0]) and is_cat(rule.rhs[1])

Now we'll create a `Grammar` class to hold a collection of rules.  We'll store the rules in maps indexed by their right-hand sides, which will facilitate lookup during parsing.  Ignore the `parse_input()` method for now — we'll define it shortly.

In [15]:
from collections import defaultdict

class Grammar:
    def __init__(self, rules=[]):
        self.lexical_rules = defaultdict(list)
        self.binary_rules = defaultdict(list)
        for rule in rules:
            add_rule(self, rule)
        print('Created grammar with %d rules.' % len(rules))
        
    def parse_input(self, input):
        """Returns a list of parses for the given input."""
        return parse_input(self, input)  # defined later

def add_rule(grammar, rule):
    if is_lexical(rule):
        grammar.lexical_rules[rule.rhs].append(rule)
    elif is_binary(rule):
        grammar.binary_rules[rule.rhs].append(rule)
    else:
        raise Exception('Cannot accept rule: %s' % rule)

Let's create a grammar using the rules we defined earlier.

In [16]:
arithmetic_grammar = Grammar(arithmetic_rules())

Created grammar with 11 rules.


Great, we have a grammar.  Now we need to implement a parsing algorithm.

### Enumerate all valid parses

The next question is, given a grammar and a specific input, how can we find the set of parses for the input which are allowed by the grammar?
<!-- TODO: Because the space of possible parses is, in general, exponential, ... -->
To solve this problem, we're going to use a variant of the [CYK algorithm][], which is an example of a [chart parsing][] algorithm, and more broadly, of [dynamic programming][].

  [CYK algorithm]: http://en.wikipedia.org/wiki/CYK_algorithm
  [chart parsing]: http://en.wikipedia.org/wiki/Chart_parser
  [dynamic programming]: http://en.wikipedia.org/wiki/Dynamic_programming

<!--Chart parsing relies on a data structure known as the *chart*, which has one entry (known as a *cell*) for every possible span in the input.  Spans are identified by a pair of token indices (*i*, *j*), where *i* is the (0-based) index of the leftmost token of the span, and *j* is one greater than the index of the rightmost token of the span.  It follows that *j* – *i* is equal to the length of the span.  For example, if the input is "one plus two", then span (0, 1) is "one", span (1, 3) is "plus two", and span (0, 3) is "one plus two".  The chart cell for each span holds a list of possible parses for that span.-->

<!--The chart parsing algorithm works like this:-->

<!--- Split the input into a sequence of tokens.
- Construct a chart, which maps from each span of the input to a list of possible parses for that span.
- Iterate over all possible spans, working bottom-up, from smaller spans to larger spans.
- For each span, and for each grammar rule, if the rule lets you build a parse for the span, add it to the chart.-->

<!--Here's how to express the algorithm in Python.  It's surprisingly simple!-->

The CYK Algorithm gives you all the possible parses, given a __grammar__.

In [17]:
from itertools import product

def parse_input(grammar, input):
    """Returns a list of all parses for input using grammar."""
    tokens = input.split()
    chart = defaultdict(list)  # map from span (i, j) to list of parses
    for j in range(1, len(tokens) + 1):
        for i in range(j - 1, -1, -1):
            apply_lexical_rules(grammar, chart, tokens, i, j)
            apply_binary_rules(grammar, chart, i, j)
    return chart[(0, len(tokens))]  # return all parses for full span

def apply_lexical_rules(grammar, chart, tokens, i, j):
    """Add parses to span (i, j) in chart by applying lexical rules from grammar to tokens."""
    for rule in grammar.lexical_rules[tuple(tokens[i:j])]:
        chart[(i, j)].append(Parse(rule, tokens[i:j]))

def apply_binary_rules(grammar, chart, i, j):
    """Add parses to span (i, j) in chart by applying binary rules from grammar."""
    for k in range(i + 1, j):  # all ways of splitting the span into two subspans
        for parse_1, parse_2 in product(chart[(i, k)], chart[(k, j)]):
            for rule in grammar.binary_rules[(parse_1.rule.lhs, parse_2.rule.lhs)]:
                chart[(i, j)].append(Parse(rule, [parse_1, parse_2]))

Lastly, let's define the `Parse` class. It's a simple container class which stores the `Rule` used to build the parse and the children to which the rule was applied.  If the rule was a lexical rule, the children are just tokens; if it was a compositional rule, the children are other `Parse`s.

In [18]:
class Parse:
    def __init__(self, rule, children):
        self.rule = rule
        self.children = tuple(children[:])
        self.semantics = compute_semantics(self)  # Ignore this for now -- we'll use it later.
        self.score = float('NaN')                 # Ditto.
        self.denotation = None                    # Ditto.

    def __str__(self):
        return '(%s %s)' % (self.rule.lhs, ' '.join([str(c) for c in self.children]))
    
def compute_semantics(parse):
    return None                                   # We'll redefine this later.

<br><br>


We're finally ready to do some parsing!  Let's see our grammar in action, by applying it to the set of 17 examples we defined above.

In [19]:
arithmetic_grammar = Grammar(arithmetic_rules())
for example in arithmetic_examples:
    parses = parse_input(arithmetic_grammar, example.input)
    print()
    print('%-16s %s' % ('input', example.input))
    for idx, parse in enumerate(parses):
        print('%-16s %s' % ('parse %d' % idx, parse))

Created grammar with 11 rules.

input            one plus one
parse 0          ($E ($EBO ($E one) ($BinOp plus)) ($E one))

input            one plus two
parse 0          ($E ($EBO ($E one) ($BinOp plus)) ($E two))

input            one plus three
parse 0          ($E ($EBO ($E one) ($BinOp plus)) ($E three))

input            two plus two
parse 0          ($E ($EBO ($E two) ($BinOp plus)) ($E two))

input            two plus three
parse 0          ($E ($EBO ($E two) ($BinOp plus)) ($E three))

input            three plus one
parse 0          ($E ($EBO ($E three) ($BinOp plus)) ($E one))

input            three plus minus two
parse 0          ($E ($EBO ($E three) ($BinOp plus)) ($E ($UnOp minus) ($E two)))

input            two plus two
parse 0          ($E ($EBO ($E two) ($BinOp plus)) ($E two))

input            three minus two
parse 0          ($E ($EBO ($E three) ($BinOp minus)) ($E two))

input            minus three minus two
parse 0          ($E ($UnOp minus) ($E ($EBO ($E three

You should observe that all 17 examples were successfully parsed.  Moreover, the examples which exhibit ambiguity get multiple parses, exactly as expected.

Syntactic parsing provides the spine around which we'll build our semantic parsing system.  But now we need to add semantics.
<!-- TODO: reword? -->

# 3. Adding semantics

Now we need to bring semantics into the picture.  Given a parse tree, we'd like to attach a semantic representation to every node in the tree.
The semantics are determined directly by the words: the semantics for "one" is `1`, the semantics for "plus" is `+`, and so on. This is province of [lexical semantics][].

  [lexical semantics]: http://en.wikipedia.org/wiki/Lexical_semantics

As we work our way up the tree, the semantic representation for each internal node is computed from the semantics of its children, in a manner that depends on the rule that was used to combine them.  This is the province of *compositional semantics*, and it hinges on the [principle of compositionality][] (often attributed to [Gottlob Frege][]).

  [Principle of compositionality]: http://en.wikipedia.org/wiki/Principle_of_compositionality
  [Gottlob Frege]: http://en.wikipedia.org/wiki/Gottlob_Frege


<div style="text-align: center; margin: 20px 100px; padding: 10px 20px 20px 20px; background-color: #eeffdd; border-style: solid; border-color: #bbccaa; border-width: 5px">
<h3>The principle of compositionality</h3>
<p>The meaning of a compound expression is a function of the meanings of its parts and the manner of their combination.</p>
</div>

<!-- Now recall that the rules of our grammar specify valid local subtrees from which we can construct parse trees.  Just as we've added semantic attachments to our parse trees, we'll add semantic attachments to the rules of our grammar. The semantic attachment to a rule specifies how to construct the semantics for the parent (LHS) category.  For a lexical rule, the semantic attachment is simply a semantic representation (or a fragment thereof).  For a compositional rule, the semantic attachment is a function which takes the semantics of the children as input and returns the semantics for the parent. -->

Let's see how this looks in Python.  We need to redefine our rules to include semantic attachments.  For *lexical* rules, the semantic attachments directly specify semantic representations (or fragments thereof):

In [20]:
numeral_rules = [
    Rule('$E', 'one', 1),
    Rule('$E', 'two', 2),
    Rule('$E', 'three', 3),
    Rule('$E', 'four', 4),
]

operator_rules = [
    Rule('$UnOp', 'minus', '~'),
    Rule('$BinOp', 'plus', '+'),
    Rule('$BinOp', 'minus', '-'),
    Rule('$BinOp', 'times', '*'),
]

For *compositional* rules, the semantic attachments are functions which specify how to construct the semantics of the parent from the semantics of the children.  We'll define these functions using Python's `lambda` syntax, and we'll establish the convention that these lambda functions always have a single parameter called `sems`, which will contain a list of the semantic representations of the children on the RHS of the rule.

For example, consider the rule which specifies how to combine a unary operator with its argument.  We can define its semantic attachment like this:

    Rule('$E', '$UnOp $E', lambda sems: (sems[0], sems[1]))

Now, `sems[0]` refers to the semantics of the child `$UnOp`,
and `sems[1]` refers to the semantics of the child `$E`.
So this semantic attachment says: take the semantics of the child `$UnOp` and the semantics of the child `$E`, form a pair from them, and return that pair as the semantics for the parent `$E`.

For the other two rules, first we combine the semantics of the left child `$E` with the semantics of `$BinOp`, forming a semantic representation for `$EBO` which is an "incomplete" tuple:

    Rule('$EBO', '$E $BinOp', lambda sems: (sems[1], sems[0]))

This semantic representation is incomplete in the sense that it combines a binary operator with a single argument.

Then we follow through by combining the "incomplete" semantics for `$EBO` with the semantics for the right child `$E` to yield the semantic representation for the parent `$E`:

    Rule('$E', '$EBO $E', lambda sems: (sems[0][0], sems[0][1], sems[1]))

Putting everything together, we get:

In [21]:
compositional_rules = [
    Rule('$E', '$UnOp $E', lambda sems: (sems[0], sems[1])),
    Rule('$EBO', '$E $BinOp', lambda sems: (sems[1], sems[0])),
    Rule('$E', '$EBO $E', lambda sems: (sems[0][0], sems[0][1], sems[1])),
]

<br>
Now that we have rules for composition, let's write a function to construct the smenatics for a parse

In [22]:
from types import FunctionType

def compute_semantics(parse):
    if is_lexical(parse.rule) or not isinstance(parse.rule.sem, FunctionType):
        return parse.rule.sem
    else:
        return parse.rule.sem([child.semantics for child in parse.children])

# Voila!

We now have all the pieces we need to do semantic parsing.  Let's test that by parsing "two times two plus three":

In [23]:
arithmetic_grammar = Grammar(arithmetic_rules())
parses = parse_input(arithmetic_grammar, "two times two plus three")
for parse in parses:
    print()
    print(parse)
    print(parse.semantics)

Created grammar with 11 rules.

($E ($EBO ($E two) ($BinOp times)) ($E ($EBO ($E two) ($BinOp plus)) ($E three)))
('*', 2, ('+', 2, 3))

($E ($EBO ($E ($EBO ($E two) ($BinOp times)) ($E two)) ($BinOp plus)) ($E three))
('+', ('*', 2, 2), 3)


OK, it looks like it's working!

Next we'd like to evaluate the performance of our semantic grammar on our whole dataset.  The SippyCup codebase includes a number of helper functions (in [`experiment.py`](./experiment.py)) to facilitate empirical evaluations.

In [24]:
from experiment import evaluate_grammar

arithmetic_grammar = Grammar(arithmetic_rules())
evaluate_grammar(grammar=arithmetic_grammar, executor=execute, examples=arithmetic_examples)

Created grammar with 11 rules.
Evaluating on 17 examples

--------------------------------------------------------------------------------
input                              one plus one
target semantics                   ('+', 1, 1)
target denotation                  2

semantics accuracy                 1
semantics oracle accuracy          1
denotation accuracy                1
denotation oracle accuracy         1
number of parses                   1
spurious ambiguity                 0

0      0.000   semantics       +   ('+', 1, 1)
               denotation      +   2

--------------------------------------------------------------------------------
input                              one plus two
target semantics                   ('+', 1, 2)
target denotation                  3

semantics accuracy                 1
semantics oracle accuracy          1
denotation accuracy                1
denotation oracle accuracy         1
number of parses                   1
spurious ambiguity   

The gap between accuracy and oracle accuracy represents an opportunity!  If we had some way of ranking candidate parses so that correct parses were likely to appear higher in the list, then we could bring accuracy closer to oracle accuracy.  (The "oracle"  in question is one that magically knows the optimal ranking of candidate parses.)

# 4. Scoring candidate parses

So far, we've seen only a couple of cases where the parser found more than one parse for a given input.  **But in richer domains, with more complex grammars, it's not unusual to find tens, hundreds, or even thousands of parses for some inputs.**  However, the list of candidate parses returned by `parse_input()` appears in arbitrary order.  The first parse is not necessarily the best, and the best parse is not necessarily the first.  Therefore, we'd like to have some way of **ranking the candidates**, so that more plausible interpretations appear earlier in the list.

  [order of operations]: http://en.wikipedia.org/wiki/Order_of_operations

An easy way to rank candidate parses is with a linear scoring function. If $p$ is a parse, $w$ is the weight vector, and $\phi$ is the vector of feature functions, we can write this as:

  [feature functions]: http://en.wikipedia.org/wiki/Feature_(machine_learning)
  [inner product]: http://en.wikipedia.org/wiki/Dot_product

$ score(p) = \sum_i w_i \cdot \phi_i(p) $
<!-- TODO: center this? -->

Finally, we sort the candidate parses by score, so that the highest-scoring parses appear first.

We now have two problems:

  1. **Feature engineering**: defining features which help to discriminate good parses from bad.
  2. **Weight learning**: deciding what weight to assign to each feature.
  
The rest of this section will focus on weight learning.

### <span style="color:red">Question: Why might we want to avoid feature engineering?</span>

<br>

Let's define the scoring function,

In [39]:
def score(parse=None, feature_fn=None, weights=None):
    """Returns the inner product of feature_fn(parse) and weights."""
    return sum(weights[feature] * value for feature, value in feature_fn(parse).items())

And the model that accepts generates parses and scores them,

In [94]:
class Model:
    def __init__(self,
                 grammar=None,
                 feature_fn=lambda parse: defaultdict(float),
                 weights=defaultdict(float),
                 executor=None):
        self.grammar = grammar
        self.feature_fn = feature_fn
        self.weights = weights
        self.executor = executor

    def parse_input(self, input):
        parses = self.grammar.parse_input(input)
        for parse in parses:
            if self.executor:
                parse.denotation = self.executor(parse.semantics)
            parse.score = score(parse, self.feature_fn, self.weights)
        parses = sorted(parses, key=lambda parse: parse.score, reverse=True)
        return parses

And the stochastic gradient descet algorithm that we will use for weight learning,

In [43]:
def latent_sgd(model=None, examples=[], training_metric=None, T=10, eta=0.1, seed=None):
    print('Running SGD learning on %d examples with training metric: %s' % (
        len(examples), training_metric.name()))
    if seed:
        print('random.seed(%d)' % seed)
        random.seed(seed)
    model = clone_model(model)
    for t in range(T):
        random.shuffle(examples)
        num_correct = 0
        for example in examples:
            # Parse input with current weights.
            parses = model.parse_input(example.input)
            # Get the highest-scoring "good" parse among the candidate parses.
            good_parses = [p for p in parses if training_metric.evaluate(example, [p])]
            if good_parses:
                target_parse = good_parses[0]
                # Get all (score, parse) pairs.
                scores = [(p.score + cost(target_parse, p), p) for p in parses]
                # Get the maximal score.
                max_score = sorted(scores)[-1][0]
                # Get all the candidates with the max score and choose one randomly.
                predicted_parse = random.choice([p for s, p in scores if s == max_score])
                if training_metric.evaluate(example, [predicted_parse]):
                    num_correct += 1
                update_weights(model, target_parse, predicted_parse, eta)
        print('SGD iteration %d: train accuracy: %.3f' % (t, 1.0 * num_correct / len(examples)))
    print_weights(model.weights)
    return model

def cost(parse_1, parse_2):
    return 0.0 if parse_1 == parse_2 else 1.0

def clone_model(model):
    return Model(grammar=model.grammar,
                 feature_fn=model.feature_fn,
                 weights=defaultdict(float),  # Zero the weights.
                 executor=model.executor)

def update_weights(model, target_parse, predicted_parse, eta):
    target_features = model.feature_fn(target_parse)
    predicted_features = model.feature_fn(predicted_parse)
    for f in set(target_features.keys() + predicted_features.keys()):
        update = target_features[f] - predicted_features[f]
        if update != 0.0:
            # print 'update %g + %g * %g = %g\t%s' % (
            #     model.weights[f], eta, update, model.weights[f] + eta * update, f)
            model.weights[f] += eta * update

<br>
### Learning to score from semantics,

Let's put `latent_sgd()` to the test, by using it to learn weights for our arithmetic model.
We'll divide our 17 arithmetic examples into 13 training examples and 4 test examples.
Then, we'll use the utility function `train_test()`, defined in [`experiment.py`](./experiment.py). Here we go:

In [45]:
from experiment import train_test
from metrics import SemanticsAccuracyMetric

train_test(model=arithmetic_model,
           train_examples=arithmetic_examples[:13],
           test_examples=arithmetic_examples[13:],
           training_metric=SemanticsAccuracyMetric(),
           seed=1)

13 training examples, 4 test examples
Evaluating on 13 train examples

--------------------------------------------------------------------------------
Over 13 examples:

semantics accuracy                 0.846
semantics oracle accuracy          1.000
denotation accuracy                0.923
denotation oracle accuracy         1.000
number of parses                   1.154
spurious ambiguity                 0.000

Evaluating on 4 test examples

--------------------------------------------------------------------------------
Over 4 examples:

semantics accuracy                 0.750
semantics oracle accuracy          1.000
denotation accuracy                0.750
denotation oracle accuracy         1.000
number of parses                   1.250
spurious ambiguity                 0.000

Running SGD learning on 13 examples with training metric: semantics accuracy

random.seed(1)
SGD iteration 0: train accuracy: 0.846
SGD iteration 1: train accuracy: 0.846
SGD iteration 2: train accuracy: 0

Note that, while training accuracy increased from 0.846 (11 of 13 correct) to 1.000 (all 13 correct), test accuracy is only 0.750 (3 of 4 correct).  We seem to have learned *something* from the training data, but whatever we learned did not generalize to the test data.  

The one error we're making on the test data involves a precedence pair that does not appear in the training data.  If you add the argument `print_examples=True` to `train_test()`, you can see which example we're getting wrong: it's `"two times two plus three"`.  In order to get this example right, we need to know that `*` should take precedence over `+`.  But none of the examples in the training data involved this precedence pair, so we haven't learned that.

With so few training examples, it's not surprising that a test example hinges on a feature not seen in training.  Going forward, we'll strive for larger datasets which are less subject to this kind of sampling noise.

<br>
### Learning to score from denotations,

It's clear that we need larger datasets, with more training examples.  But as we noted [above](#arithmetic-example-data), annotating examples with target semantics can be slow, expensive, and error-prone, and may require expert knowledge.  However, annotating examples with target *denotations* can often be faster, cheaper, and more reliable.

### <span style="color:red">Question: Why is it easier to get target denotations rather than target sematics? </span>

<!-- The arithmetic domain illustrates this nicely.  Writing down the target semantics for `"minus three minus two"` (namely, `('-', ('~', 3), 2)`) is a tedious chore that most people probably could not perform reliably.  You need to understand Lisp-y prefix notation.  You need to remember to use the funny `'~'`, instead of the more natural `'-'`, for unary `"minus"`.  You need to remember to quote the operators.  And you need to get the order of operations right.  By contrast, writing down the target denotation (namely, `-5`) is easy as pie.  The only thing you really need to think about is the order of operations, which most people are capable of mastering.  So if you're asking only for denotations, rather than semantics, you can get more annotations, faster, cheaper, and more reliably, from ordinary people. -->


<br>
**One of the principal contributions of [Liang et al. 2011][] was to show that it is possible to learn scoring models for semantic parsing using only target denotations, rather than target semantics, as the source of supervision.  The central idea is presented with admirable clarity in [Liang & Potts 2015].**

  [Liang et al. 2011]: http://www.cs.berkeley.edu/~jordan/papers/liang-jordan-klein-acl2011.pdf
  [Liang & Potts 2015]: http://www.annualreviews.org/doi/pdf/10.1146/annurev-linguist-030514-125312

In SippyCup, we can change the source of supervision from semantics to denotations simply by changing the training metric from `SemanticsAccuracyMetric` to `DenotationAccuracyMetric`.  With this change, a parse will count as correct iff the denotation of its semantic yield matches the target denotation in the example.

Let's begin by repeating the experiment we did a moment ago, but switching to denotations as the source of supervision:

In [25]:
from metrics import DenotationAccuracyMetric

train_test(model=arithmetic_model,
           train_examples=arithmetic_examples[:13],
           test_examples=arithmetic_examples[13:],
           training_metric=DenotationAccuracyMetric(),
           seed=1)

NameError: name 'train_test' is not defined

Now that we're learning from denotations, the semantics accuracy on the training data did not reach 1.000, as it did before.  One of the [exercises](#arithmetic-exercises) will ask you to investigate why.  However, the denotation accuracy did reach 1.000.

### <span style="color:red"> Question: Why don't the test set results budge? </span>



<br>
<br>

The file [`arithmetic.py`](./arithmetic.py) contains a set of 100 "development" examples for the arithmetic domain.  These examples are annotated only with denotations, not with semantics.  100 examples is still not huge, but the arithmetic domain is simple enough that it should suffice.

In [47]:
from arithmetic import arithmetic_dev_examples
from metrics import denotation_match_metrics

train_test(model=arithmetic_model,
           train_examples=arithmetic_dev_examples,
           test_examples=arithmetic_examples[13:],
           metrics=denotation_match_metrics(),
           training_metric=DenotationAccuracyMetric(),
           seed=1)

100 training examples, 4 test examples
Evaluating on 100 train examples

--------------------------------------------------------------------------------
Over 100 examples:

denotation accuracy                0.640
denotation oracle accuracy         1.000
number of parses                   4.520
spurious ambiguity                 0.000

Evaluating on 4 test examples

--------------------------------------------------------------------------------
Over 4 examples:

denotation accuracy                0.750
denotation oracle accuracy         1.000
number of parses                   1.250
spurious ambiguity                 0.000

Running SGD learning on 100 examples with training metric: denotation accuracy

random.seed(1)
SGD iteration 0: train accuracy: 0.670
SGD iteration 1: train accuracy: 0.850
SGD iteration 2: train accuracy: 0.890
SGD iteration 3: train accuracy: 0.900
SGD iteration 4: train accuracy: 0.900
SGD iteration 5: train accuracy: 0.910
SGD iteration 6: train accuracy: 0.91

Note that denotation accuracy on the test examples now reaches 1.000 after training.  The sole error is corrected, because the larger training dataset affords ample opportunity to learn that `*` should take precedence over `+`.

<br><br>

# <span style="color:red"> Exercises! </span> <a id="arithmetic-exercises"></a>

<!-- TODO: consider ordering. -->

Note that some of these exercises ask you to extend the grammar in ways that will be a bit awkward, given the requirement that rules be in Chomsky normal form (CNF).   Your solutions to these exercises should adhere to the CNF restriction.  It's going to be a bit of a pain in the ass, and that's part of the point.

### Straightforward

1) How many parses do you get for "one plus one plus one plus one plus one"?  Why? 

In [2]:
## ADD CODE HERE ##
# Hint: look at parse_input


2) Extend the grammar to support postfix unary operators, as in "two squared" or "two cubed".

In [117]:
operator_rules = [
    Rule('$UnOp', 'minus', '~'),
    Rule('$BinOp', 'plus', '+'),
    Rule('$BinOp', 'minus', '-'),
    Rule('$BinOp', 'times', '*'),
    ## ADD CODE HERE ##
]

arithmetic_grammar = Grammar(arithmetic_rules())

Created grammar with 13 rules.


3) Extend the grammar to support divison, as in "four divided by three" or "minus four over two".

In [118]:
operator_rules = [
    Rule('$UnOp', 'minus', '~'),
    Rule('$BinOp', 'plus', '+'),
    Rule('$BinOp', 'minus', '-'),
    Rule('$BinOp', 'times', '*'),
    ## ADD CODE HERE ##
]

arithmetic_grammar = Grammar(arithmetic_rules())

Created grammar with 15 rules.


4) Extend the grammar to support multi-word operators, as in "the square root of one" or "the average of one and two".  (You may need to extend the semantic representation and the executor as well.)

In [1]:
operator_rules = [
    Rule('$UnOp', 'minus', '~'),
    Rule('$BinOp', 'plus', '+'),
    Rule('$BinOp', 'minus', '-'),
    Rule('$BinOp', 'times', '*'),
    ## ADD CODE HERE ##
]

arithmetic_grammar = Grammar(arithmetic_rules())

sems = [
    ('+', 1, 1),                # one plus one
    ('-', ('~', 3), 2),         # minus three minus two
    ('-', ('+', 3, 3), 2),      # three plus three minus two
    ('+', ('*', 2, 2), 3),      # two times two plus three
    ## ADD CODE HERE ##
]

ops = {
    '~': lambda x: -x,
    '+': lambda x, y: x + y,
    '-': lambda x, y: x - y,
    '*': lambda x, y: x * y,
    ## ADD CODE HERE ##
}


for sem in sems:
    print('{} = {}'.format(sem, execute(sem)))

NameError: name 'Rule' is not defined

5) When we learned from semantics, semantics accuracy on the training examples reached 1.000 after training.  However, when we switched to learning from denotations, it did not.  Explain why.  Be precise and specific.

### Straight Forward (do this if you have time)

6) When we trained on the 100 examples in `arithmetic_dev_examples`, denotation accuracy on the training examples did not reach 1.000 after training.  Diagnose the errors and describe your findings.  What could help us to eliminate those errors?  (You don't need to implement the fix.  Just give a precise diagnosis.)

Copyright (C) 2015 Bill MacCartney